In [1]:
!pip install wandb
!pip install xtarfile
!pip install pyenchant

In [2]:
START_TOKEN="\t"
END_TOKEN="\n"

In [3]:
import os
import random
import time
import wandb
import re, string
import numpy as np
import pandas as pd 
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
from wordcloud import WordCloud, STOPWORDS
from collections import Counter


import os
from os.path import exists
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random 
import numpy as np
import keras
import tensorflow as tf
import csv

from tensorflow.keras import layers
import tensorflow.keras.backend as K

In [4]:

from Utility.Encoder  import Encoder
from Utility.Decoder  import Decoder
from Utility.Attention  import BahdanauAttention
from Utility.SeqTOSeq  import SequenceTOSequence
from Utility.Param  import Parameters
from Utility.DataLoader import downloadDataSet,get_files,tokenize,preprocess_data

# Data Preprocessing

In [5]:
downloadDataSet()

You are all set


In [6]:
language="te"
train_dir, val_dir, test_dir = get_files(language)

dataset, input_tokenizer, targ_tokenizer = preprocess_data(train_dir)
val_dataset, _, _ = preprocess_data(val_dir)


In [7]:
#train data 
dataset, input_tokenizer, targ_tokenizer = preprocess_data(train_dir)

## Train Model

In [12]:

def run_model_on_test_dataset(model,param,test_dir):

    ## Character level accuracy ##
    test_dataset, _, _ = preprocess_data(test_dir, model.input_tokenizer, model.targ_tokenizer)
    test_loss, test_acc = model.evaluate(test_dataset, batch_size=100)
    print('Character level accuracy: '+str(test_acc.numpy()))

    ##  Word level accuracy ##
    test_tsv = pd.read_csv(test_dir, sep="\t", header=None)
    inputs = test_tsv[1].astype(str).tolist()
    targets = test_tsv[0].astype(str).tolist()
   
    outputs = []

    for word in inputs:
        outputs.append(model.translate(word)[0])

    print(f"Word level accuracy: {np.sum(np.asarray(outputs) == np.array(targets)) / len(outputs)}")

    if param.save_outputs is not None:
        df = pd.DataFrame()
        df["inputs"] = inputs
        df["targets"] = targets
        df["outputs"] = outputs
        df.to_csv(param.save_outputs)


    return inputs,targets,outputs

#randomly_evaluate(model, n=15)

In [9]:
param=Parameters(language="te",\
                        embedding_dim=128,\
                        encoder_layers=2,\
                        decoder_layers=2,\
                        layer_type="lstm",\
                        units=256,\
                        dropout=0.5,
                        epochs=1,\
                 batch_size=128,\
                 attention=True,\
                 restoreBestModel=True,\
                 patience=2\
                   )

In [10]:
model = SequenceTOSequence(param)
model.set_vocabulary(input_tokenizer, targ_tokenizer)
model.build(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
                metric = tf.keras.metrics.SparseCategoricalAccuracy())

model.fit(dataset, val_dataset, epochs=param.epochs, wandb=param.wandb, teacher_forcing_ratio=0)                  

Epochs : 100%|██████████| 1/1 [01:42<00:00, 102.41s/it]


Train Loss: 1.3618 Train Accuracy: 59.7964 Validation Loss: 1.7215 Validation Accuracy: 54.7164

Time taken for the epoch 102.4054

Model trained successfully !!


In [13]:
inputs,targets,outputs=run_model_on_test_dataset(model,param,test_dir)


Running test dataset through the model...

Test Loss: 1.3318 Test Accuracy: 0.6070
Character level accuracy: 0.6070272
Word level accuracy: 0.0


## Evaluate Model

In [14]:

def evaluate_samples(model, n=0):

    df = pd.read_csv(test_dir, sep="\t", header=None)

    printLine=True
    source=[]
    actual=[]
    predict=[]
    if(n==0):
      print(f"Evaluating the model on all words\n")
      n=len(df)
      printLine=False
    else:
      df = df.sample(n=n).reset_index(drop=True)
      #print(f"Randomly evaluating the model on {n} words\n")

    for i in range(n):
        word = str(df[1][i])
        source.append(word)
        actual.append(str(df[0][i]))
        predict.append(model.translate(word)[0])
        """if(printLine):
          print(f"Input word: {word}")
          print(f"Actual translation: {str(df[0][i])}")
          print(f"Model translation: {predict[-1]}\n")"""
    return source,actual,predict


In [16]:
source,actual,predict=inputs,targets,outputs

NameError: ignored

## Report Results

In [ ]:
from IPython.display import HTML as html_print
from IPython.display import display
def compare_result(predict,actual,word):
    string1=list(actual)
    string2=list(predict)
    maxstr=None
    
    minstr=None
    if(len(string1)>len(string2)):
      maxstr=string1
      minstr=string2
    else:
      maxstr=string2
      minstr=string1
   
    shift=0
    html_str = """
    <br>
      <table style="border:2px solid black; border-collapse:collapse">
      <tr>
      <td scope="row" style="border:1px solid black;background-color:#3498DB;color:white;padding:10px;text-align:left"> <strong>{}</strong> </td>
      """.format(actual)
    maxlen=len(maxstr)
    minlen=len(minstr)
    char_html=""
    j=0
    for i in range(maxlen):
    
        if(j<minlen):
          if(maxstr[i]==minstr[j]):
              char_html+="""<td scope="row" style="background-color:#E3FFE7 ;border:1px solid black;padding:15px;text-align:left"> <strong>{}</strong> </td>""".format(maxstr[i] )
              j+=1
          else:
             if(i+1<maxlen and  maxstr[i+1]==minstr[j]):
                char_html+=""" <td scope="row" style="background-color:#FFF6F7 ; border:1px solid black;padding:15px;text-align:left"> <strong>{}</strong> </td>""".format(maxstr[i] )
                continue
             else:
              j+=1
              char_html+=""" <td scope="row" style="background-color:#FFF6F7 ; border:1px solid black;padding:15px;text-align:left"> <strong>{}</strong> </td>""".format(maxstr[i] )
        else:
          char_html+="""<td scope="row" style="background-color: #FFF6F7 ; border:1px solid black;padding:15px;text-align:left"> <strong>{} </strong></td>""".format(maxstr[i] )   

    char_html+="""</tr>"""
    char_html+="""<tr>
      <td scope="row" style="border:1px solid black;background-color:#3498DB;padding:15px;color:white;text-align:left"> <strong>{} </strong></td>""".format(predict)

    j=0
    for i in range(maxlen):

        if(j<minlen):
        
          if(maxstr[i]==minstr[j]):
            char_html+="""<td scope="row" style="background-color:#E3FFE7 ;border:1px solid black;padding:15px;text-align:left"><strong> {} </strong></td>""".format(minstr[j] )
            j+=1
          else:
             if(i+1<maxlen and  maxstr[i+1]==minstr[j]):
                char_html+="""<td scope="row" style="background-color:#FFF6F7 ; border:1px solid black;padding:15px;text-align:left"> <strong>{}</strong> </td>""".format(' ' )
                continue
             else:
                
                char_html+="""<td scope="row" style="background-color:#FFF6F7 ; border:1px solid black;padding:15px;text-align:left"> <strong>{}</strong> </td>""".format(minstr[j] )
                j+=1
        else:
          char_html+="""<td scope="row" style="background-color:#FFF6F7  ; border:1px solid black;padding:15px;text-align:left"><strong> {}</strong> </td>""".format('' )



    char_html+="""</tr>"""

    char_html+="""<tr>
      <td scope="row" style="border:1px solid black;background-color:#3498DB;padding:15px;color:white;text-align:left"> <strong>{} </strong></td>""".format(word)


    j=0
    for i in range(maxlen):
    
        if(j<minlen):
          if(maxstr[i]==minstr[j]):
              char_html+="""<td scope="row" style="border:1px solid black;padding:15px;text-align:left">&#x2705; </td>"""
              j+=1
          else:
              if(i+1<maxlen and  maxstr[i+1]==minstr[j]):
                char_html+="""<td scope="row" style="border:1px solid black;padding:15px;text-align:left"> &#x274C; </td>""" 
                continue
              else:
                j+=1
                char_html+=""" <td scope="row" style=" border:1px solid black;padding:15px;text-align:left"> &#x274C; </td>"""
        else:
          char_html+="""<td scope="row" style="border:1px solid black;padding:15px;text-align:left"> &#x274C; </td>""" 


    char_html+="""</tr>"""
    html_str+=char_html
    display(html_print(html_str)) 

In [ ]:
maxlengthword=0
for i in actual:
  if(len(i)>maxlengthword):
    maxlengthword=len(i)


In [ ]:
source,actual,predict
total_words=np.zeros(maxlengthword+1)
total_correct=np.zeros(maxlengthword+1)
total_wrong=np.zeros(maxlengthword+1)

html_str = """
    <br>
      <table style="border:2px solid black; border-collapse:collapse">
      <tr>
      <th scope="row" style="border:1px solid black;background-color:#3498DB;color:white;padding:10px;text-align:left"> <strong>Word Length</strong> </th>
      <th scope="row" style="border:1px solid black;background-color:#3498DB;color:white;padding:10px;text-align:left"> <strong>Total Words</strong> </th>
      <th scope="row" style="border:1px solid black;background-color:#3498DB;color:white;padding:10px;text-align:left"> <strong>Accuracy</strong> </th></tr>
      """
for (i,j) in zip(actual,predict):
  total_words[len(i)]+=1
  if(i==j):
    total_correct[len(i)]+=1
  else:
    total_wrong[len(i)]+=1

for i in range(maxlengthword):
  
  if(total_words[i]!=0):
    html_str+=""" <td scope="row" style="border:1px solid black;padding:10px;text-align:left"> <strong>{}</strong> </th>
      <th scope="row" style="border:1px solid black;padding:10px;text-align:left"> <strong>{}</strong> </th>
      <th scope="row" style="border:1px solid black;padding:10px;text-align:left"> <strong>{}</strong> </th></tr>
      """.format( str(i),str(int(total_words[i])),str("{:.2f}".format(total_correct[i]*100/total_words[i])))
html_str += """</tr></table>"""

display(html_print(html_str)) 


In [ ]:
source_rnd,actual_rnd,predict_rnd=model.evaluate_samples(model,10)

for i,j,k in zip(source_rnd,actual_rnd,predict_rnd):
    compare_result(j,k,i)